In [73]:
import numpy as np
from scipy import stats
import scipy.optimize as opt
import converge
import numerical

In [50]:
pdf = stats.multivariate_normal((23, 7), 4).pdf
fun = lambda x: -pdf(x)

In [52]:
opt.minimize(fun, [1, 1]).x

array([1, 1])

In [48]:
import scipy.optimize as opt
from converge import Converge
from numpy.linalg import inv

def minimize(f, theta, g, H, epsilon = 0.0000001, maxsteps = 100):
    converge = Converge(f, epsilon, maxsteps)
    
    while not converge.done(theta):
        d = -inv(H(theta)).dot(g(theta))
        theta += d * opt.minimize(lambda eta: f(theta + eta * d), 1).x
    return theta

In [76]:
numerical.grad(lambda x: (x - 3) ** 2 , 3.5)

array(1.0000000000065512)

In [ ]:
x = T.dvector('x')
y = x ** 2
cost = y.sum()
gy = T.grad(cost, x)
H, updates = theano.scan(lambda i, gy,x : T.grad(gy[i], x), sequences=T.arange(gy.shape[0]), non_sequences=[gy, x])
f = theano.function([x], H, updates=updates)